In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter

<function warnings.simplefilter(action, category=<class 'Warning'>, lineno=0, append=False)>

# Content Based Filtering

In this section we find similar movies by comparing the features of each movie. To do this we will utilize thr Bag of Words technique and then convert each word to a vector using CountVectorization. We chose count vectorization because we only looking at the existence of a word in each movie's features.Our bag of words will include, genres, actors and director. Plot was excluded because we are not conducting sentiment analysis.

In [2]:
# load dataset

movies = pd.read_csv('Data/movies_sml.csv')
movies.head()

,movieId,Title,genres,Actors,Director,Plot,Poster
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",John Lasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Jumanji,Adventure|Children|Fantasy,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Grumpier Old Men,Comedy|Romance,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Howard Deutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Waiting to Exhale,Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Father of the Bride Part II,Comedy,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


### Movies Dataset Preprocessing

#### Removing spaces from actors and director names

The spaces are removed so that only full names are considered in the search. For e.g actor Tom Hanks becomes TomHanks. If we don't remove the space, then an occurrence of another actor with the first name tom will produce a false similarity.

In [3]:
# function to switch all words to lower and remove spaces
def clean_data(x):
    if isinstance(x, list):
        return[str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [4]:
credits = ['Actors', 'Director']

for credit in credits:
    movies[credit] = movies[credit].apply(clean_data)

In [5]:
# remove | from genres

movies['genres'] = movies['genres'].str.replace("|", " ")

In [6]:
movies.head()

,movieId,Title,genres,Actors,Director,Plot,Poster
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,"tomhanks,timallen,donrickles,jimvarney",johnlasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Jumanji,Adventure Children Fantasy,"robinwilliams,jonathanhyde,kirstendunst,bradle...",joejohnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Grumpier Old Men,Comedy Romance,"waltermatthau,jacklemmon,sophialoren,ann-margret",howarddeutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Waiting to Exhale,Comedy Drama Romance,"whitneyhouston,angelabassett,lorettadevine,lel...",forestwhitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Father of the Bride Part II,Comedy,"stevemartin,dianekeaton,martinshort,kimberlywi...",charlesshyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


#### Combining data 

In [7]:
def create_soup(x):
    return  x['Actors'].replace(',',' ') + ' ' + x['Director'] + ' ' + ' '.join(x['genres'])

In [8]:
movies['bag_of_words'] = movies.apply(create_soup, axis=1)

In [9]:
movies.head()

,movieId,Title,genres,Actors,Director,Plot,Poster,bag_of_words
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,"tomhanks,timallen,donrickles,jimvarney",johnlasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,tomhanks timallen donrickles jimvarney johnlas...
1,2,Jumanji,Adventure Children Fantasy,"robinwilliams,jonathanhyde,kirstendunst,bradle...",joejohnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...,robinwilliams jonathanhyde kirstendunst bradle...
2,3,Grumpier Old Men,Comedy Romance,"waltermatthau,jacklemmon,sophialoren,ann-margret",howarddeutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...,waltermatthau jacklemmon sophialoren ann-margr...
3,4,Waiting to Exhale,Comedy Drama Romance,"whitneyhouston,angelabassett,lorettadevine,lel...",forestwhitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...,whitneyhouston angelabassett lorettadevine lel...
4,5,Father of the Bride Part II,Comedy,"stevemartin,dianekeaton,martinshort,kimberlywi...",charlesshyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...,stevemartin dianekeaton martinshort kimberlywi...


In [10]:
movies_bow = movies[['Title', 'movieId', 'bag_of_words']]
movies_bow.to_csv('Data/movies_bow.csv', index=False)
movies_bow.head()

,Title,movieId,bag_of_words
0,Toy Story,1,tomhanks timallen donrickles jimvarney johnlas...
1,Jumanji,2,robinwilliams jonathanhyde kirstendunst bradle...
2,Grumpier Old Men,3,waltermatthau jacklemmon sophialoren ann-margr...
3,Waiting to Exhale,4,whitneyhouston angelabassett lorettadevine lel...
4,Father of the Bride Part II,5,stevemartin dianekeaton martinshort kimberlywi...


In [15]:
# send to csv for use in Flask App

movies_bow = pd.read_csv('movies_bow.csv')

### Building the recommender system 
#### Importing sklearn moules

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### Initializing and generating the count matrix 

In [17]:
count = CountVectorizer()
count_matrix = count.fit_transform(movies_bow['bag_of_words'])

#### Generating the cosine similarity matrix

In [18]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

#### Function that Produces Recommendations

In [19]:
#map index to Title. This will be used in the function and is integral to 
# how we access the recommended movies

indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

#Create recommendation function

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:26]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_bow['Title'].iloc[movie_indices]

In [20]:
# test one movie

get_recommendations('Rumble in the Bronx')

11663                   Kung Fu Yoga
4129                       Who Am I?
3463                     Crime Story
670                       Supercop 2
690      Around the World in 80 Days
691      Around the World in 80 Days
1143                       Mr. Magoo
1416                       Rush Hour
1595                      Black Mask
2008                   Shanghai Noon
2440                     Rush Hour 2
4071                     City Hunter
4976                     Thunderbolt
5320                     Rush Hour 3
9890                    Dragon Blade
11352                      Skiptrace
11463                Railroad Tigers
1226                    Mr. Nice Guy
3052                Shanghai Knights
4397                        Gorgeous
4649                New Police Story
6441               Shinjuku Incident
6894              Little Big Soldier
4974                    Swordsman II
0                          Toy Story
Name: Title, dtype: object